In [ ]:
import unittest
from unittest import mock
from unittest.mock import Mock, patch, AsyncMock
from http import HTTPStatus
import os

from your_package.utils.conversation_utils import ConversationManagementUtils


class TestConversationManagementUtils(unittest.TestCase):
    def setUp(self):
        self.app_id = "test_app"
        self.request_handler = Mock()
        self.utils = ConversationManagementUtils(
            app_id=self.app_id,
            request_handler=self.request_handler,
            semaphore_limit=10
        )
    
    def test_create_headers(self):
        headers = self.utils._create_headers("test_sso")
        self.assertEqual(headers["Cookie"], "SSO=test_sso")
        self.assertEqual(headers["app-id"], self.app_id)
    
    def test_get_all_conversations_success(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.return_value = [
            {"conversationId": "conv1"},
            {"conversationId": "conv2"}
        ]
        self.request_handler.get.return_value = mock_response
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, ["conv1", "conv2"])
        
        self.request_handler.get.assert_called_once()
        args, kwargs = self.request_handler.get.call_args
        self.assertTrue(kwargs["path"].endswith("/conversations"))
    
    def test_get_all_conversations_failure(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.BAD_REQUEST
        mock_response.text = "Error text"
        self.request_handler.get.return_value = mock_response
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    def test_get_all_conversations_exception(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.side_effect = Exception("Test error")
        self.request_handler.get.return_value = mock_response
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    def test_get_all_conversations_none_response(self):
        self.request_handler.get.return_value = None
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    async def test_async_get_all_conversations_success(self):
        mock_response = Mock()
        mock_response.status = HTTPStatus.OK
        mock_response.json = AsyncMock(return_value=[
            {"conversationId": "conv1"},
            {"conversationId": "conv2"}
        ])
        self.request_handler.async_get = AsyncMock(return_value=mock_response)
        
        result = await self.utils.async_get_all_conversations("test_sso")
        self.assertEqual(result, ["conv1", "conv2"])
    
    async def test_async_get_all_conversations_failure(self):
        mock_response = Mock()
        mock_response.status = HTTPStatus.BAD_REQUEST
        mock_response.text = AsyncMock(return_value="Error text")
        self.request_handler.async_get = AsyncMock(return_value=mock_response)
        
        result = await self.utils.async_get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    async def test_async_get_all_conversations_exception(self):
        mock_response = Mock()
        mock_response.status = HTTPStatus.OK
        mock_response.json = AsyncMock(side_effect=Exception("Test error"))
        self.request_handler.async_get = AsyncMock(return_value=mock_response)
        
        result = await self.utils.async_get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    def test_get_conversation_id_success(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.return_value = {
            "conversationId": "conv123",
            "data": "test_data"
        }
        self.request_handler.post.return_value = mock_response
        
        conv_id, data = self.utils.get_conversation_id("test_sso", {"pref": "value"})
        self.assertEqual(conv_id, "conv123")
        self.assertEqual(data["data"], "test_data")
        
        args, kwargs = self.request_handler.post.call_args
        self.assertEqual(kwargs["payload"]["preferences"], {"pref": "value"})
    
    def test_get_conversation_id_failure(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.BAD_REQUEST
        mock_response.text = "Error text"
        self.request_handler.post.return_value = mock_response
        
        conv_id, data = self.utils.get_conversation_id("test_sso")
        self.assertIsNone(conv_id)
        self.assertEqual(data, {})
    
    def test_get_conversation_id_exception(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.side_effect = Exception("Test error")
        self.request_handler.post.return_value = mock_response
        
        conv_id, data = self.utils.get_conversation_id("test_sso")
        self.assertIsNone(conv_id)
        self.assertEqual(data, {})
    
    @patch('builtins.open', new_callable=mock.mock_open, read_data=b'test file content')
    def test_upload_success(self, mock_open):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.return_value = {"document_id": "doc123"}
        self.request_handler.post.return_value = mock_response
        
        result = self.utils.upload("test_sso", "test/path.pdf")
        self.assertEqual(result, "doc123")
        
        mock_open.assert_called_once_with("test/path.pdf", "rb")
        args, kwargs = self.request_handler.post.call_args
        self.assertIn("files", kwargs)
    
    def test_upload_unsupported_file_type(self):
        result = self.utils.upload("test_sso", "test/path.unsupported")
        self.assertIsNone(result)
        self.request_handler.post.assert_not_called()
    
    @patch('builtins.open', new_callable=mock.mock_open, read_data=b'test file content')
    def test_upload_api_error(self, mock_open):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.BAD_REQUEST
        mock_response.text = "Error text"
        self.request_handler.post.return_value = mock_response
        
        result = self.utils.upload("test_sso", "test/path.pdf")
        self.assertIsNone(result)
    
    @patch('builtins.open', side_effect=IOError("File not found"))
    def test_upload_file_error(self, mock_open):
        result = self.utils.upload("test_sso", "nonexistent/path.pdf")
        self.assertIsNone(result)
        self.request_handler.post.assert_not_called()


class AsyncMock(Mock):
    async def __call__(self, *args, **kwargs):
        return super(AsyncMock, self).__call__(*args, **kwargs)


if __name__ == "__main__":
    unittest.main()

ModuleNotFoundError: No module named 'your_package'

In [ ]:
"""
Utility functions for working with Gemini API formats.
"""

import inspect
import json
from typing import Dict, List, Any, Optional, Callable, Union, get_type_hints, get_origin, get_args
from loguru import logger


class GeminiUtils:
    """
    Utilities for working with Gemini API formats and conversions.
    """
    
    class ParameterBuilder:
        """
        Builder class for creating parameter schemas for Gemini function calling.
        """
        
        @staticmethod
        def string(description: Optional[str] = None) -> Dict[str, Any]:
            """Create a string parameter schema."""
            schema = {"type": "STRING"}
            if description:
                schema["description"] = description
            return schema
        
        @staticmethod
        def integer(description: Optional[str] = None) -> Dict[str, Any]:
            """Create an integer parameter schema."""
            schema = {"type": "INTEGER"}
            if description:
                schema["description"] = description
            return schema
        
        @staticmethod
        def number(description: Optional[str] = None) -> Dict[str, Any]:
            """Create a number parameter schema."""
            schema = {"type": "NUMBER"}
            if description:
                schema["description"] = description
            return schema
        
        @staticmethod
        def boolean(description: Optional[str] = None) -> Dict[str, Any]:
            """Create a boolean parameter schema."""
            schema = {"type": "BOOLEAN"}
            if description:
                schema["description"] = description
            return schema
        
        @staticmethod
        def array(items_schema: Dict[str, Any], description: Optional[str] = None) -> Dict[str, Any]:
            """
            Create an array parameter schema.
            
            Args:
                items_schema: Schema for the array items
                description: Optional description
                
            Returns:
                Array parameter schema
            """
            schema = {
                "type": "ARRAY",
                "items": items_schema
            }
            if description:
                schema["description"] = description
            return schema
        
        @staticmethod
        def object(
            properties: Dict[str, Dict[str, Any]],
            required: Optional[List[str]] = None,
            description: Optional[str] = None
        ) -> Dict[str, Any]:
            """
            Create an object parameter schema.
            
            Args:
                properties: Dictionary of property schemas
                required: List of required property names
                description: Optional description
                
            Returns:
                Object parameter schema
            """
            schema = {
                "type": "OBJECT",
                "properties": properties
            }
            if required:
                schema["required"] = required
            if description:
                schema["description"] = description
            return schema
        
        @staticmethod
        def enum(values: List[str], description: Optional[str] = None) -> Dict[str, Any]:
            """
            Create an enum parameter schema.
            
            Args:
                values: List of allowed values
                description: Optional description
                
            Returns:
                Enum parameter schema
            """
            schema = {
                "type": "STRING",
                "enum": values
            }
            if description:
                schema["description"] = description
            return schema
    
    @staticmethod
    def create_function_schema(
        name: str,
        description: str,
        parameters: Dict[str, Dict[str, Any]],
        required_parameters: Optional[List[str]] = None,
        response_schema: Optional[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """
        Create a function schema for Gemini API.
        
        Args:
            name: Function name
            description: Function description
            parameters: Dictionary of parameter schemas
            required_parameters: List of required parameter names
            response_schema: Optional schema for the function response
            
        Returns:
            Function schema in Gemini API format
        """
        function_def = {
            "name": name,
            "description": description,
            "parameters": {
                "type": "OBJECT",
                "properties": parameters
            }
        }
        
        # Add required parameters if provided
        if required_parameters:
            function_def["parameters"]["required"] = required_parameters
        
        # Add response schema if provided
        if response_schema:
            function_def["response"] = response_schema
            
        return function_def
    
    @staticmethod
    def pydantic_to_schema(schema: Any) -> Dict[str, Any]:
        """
        Convert Pydantic schema to Gemini API schema format.
        
        Args:
            schema: Pydantic schema to convert
            
        Returns:
            Schema in Gemini API format
        """
        if hasattr(schema, "model_json_schema"):
            json_schema = schema.model_json_schema()
            
            # Convert JSON schema to Gemini API format
            gemini_schema = {
                "type": "OBJECT",
                "properties": {}
            }
            
            for prop_name, prop_info in json_schema.get("properties", {}).items():
                prop_type = prop_info.get("type")
                
                if prop_type == "string":
                    gemini_schema["properties"][prop_name] = {"type": "STRING"}
                elif prop_type == "integer":
                    gemini_schema["properties"][prop_name] = {"type": "INTEGER"}
                elif prop_type == "number":
                    gemini_schema["properties"][prop_name] = {"type": "NUMBER"}
                elif prop_type == "boolean":
                    gemini_schema["properties"][prop_name] = {"type": "BOOLEAN"}
                elif prop_type == "array":
                    gemini_schema["properties"][prop_name] = {"type": "ARRAY"}
                elif prop_type == "object":
                    gemini_schema["properties"][prop_name] = {"type": "OBJECT"}
            
            # Add required fields
            if "required" in json_schema:
                gemini_schema["required"] = json_schema["required"]
                
            return gemini_schema
        
        # If not a Pydantic model, return as is (assuming it's already in correct format)
        return schema
    
    @staticmethod
    def python_to_function(
        function: Callable,
        name: Optional[str] = None,
        description: Optional[str] = None,
        schema: Optional[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """
        Convert Python function to Gemini API function declaration format.
        
        Args:
            function: Function to convert
            name: Optional custom name
            description: Optional custom description
            schema: Optional user-provided schema (overrides automatic extraction)
            
        Returns:
            Function declaration in Gemini API format
        """
        # If schema is provided, use it (with function name/description override)
        if schema:
            # Get function metadata for override
            func_name = name or function.__name__
            func_description = description or inspect.getdoc(function) or ""
            
            # Start with user schema
            function_def = schema.copy()
            
            # Override name and description if provided
            function_def["name"] = func_name
            if func_description:
                function_def["description"] = func_description
                
            return function_def
        
        # Get function metadata
        func_name = name or function.__name__
        func_description = description or inspect.getdoc(function) or ""
        
        # Get signature and type hints
        sig = inspect.signature(function)
        type_hints = get_type_hints(function)
        
        # Create parameters schema
        parameters = {
            "type": "OBJECT",
            "properties": {},
            "required": []
        }
        
        # Process parameters
        for param_name, param in sig.parameters.items():
            # Skip self for methods
            if param_name == "self":
                continue
                
            # Get parameter type
            param_type = type_hints.get(param_name, str)
            param_schema = GeminiUtils._python_type_to_schema(param_type)
            
            # Add to properties
            parameters["properties"][param_name] = param_schema
            
            # If no default value, it's required
            if param.default == inspect.Parameter.empty:
                parameters["required"].append(param_name)
        
        # Create function definition
        function_def = {
            "name": func_name,
            "description": func_description,
            "parameters": parameters
        }
        
        # Add return type if available
        if "return" in type_hints:
            return_type = type_hints["return"]
            if return_type is not None:
                function_def["response"] = GeminiUtils._python_type_to_schema(return_type)
        
        return function_def
    
    @staticmethod
    def _python_type_to_schema(python_type: Any) -> Dict[str, Any]:
        """
        Convert Python type to Gemini schema type.
        
        Args:
            python_type: Python type annotation
            
        Returns:
            Schema type in Gemini API format
        """
        # Handle primitive types
        if python_type == str:
            return {"type": "STRING"}
        elif python_type == int:
            return {"type": "INTEGER"}
        elif python_type == float:
            return {"type": "NUMBER"}
        elif python_type == bool:
            return {"type": "BOOLEAN"}
        
        # Handle Union/Optional types
        origin = get_origin(python_type)
        if origin is Union:
            args = get_args(python_type)
            # Handle Optional (Union with None)
            if type(None) in args:
                non_none_args = [arg for arg in args if arg is not type(None)]
                if len(non_none_args) == 1:
                    base_type = GeminiUtils._python_type_to_schema(non_none_args[0])
                    base_type["nullable"] = True
                    return base_type
            
            # General Union as anyOf
            schemas = [GeminiUtils._python_type_to_schema(arg) for arg in args 
                      if arg is not type(None)]
            return {"anyOf": schemas}
        
        # Handle List
        if origin is list or python_type == List:
            args = get_args(python_type)
            item_type = str  # Default item type
            if args:
                item_type = args[0]
            return {
                "type": "ARRAY",
                "items": GeminiUtils._python_type_to_schema(item_type)
            }
        
        # Handle Dict
        if origin is dict or python_type == Dict:
            return {"type": "OBJECT"}
        
        # Default to string for unknown types
        return {"type": "STRING"}
    
    @staticmethod
    def functions_to_tools(functions: List[Union[Callable, Dict[str, Any]]]) -> List[Dict[str, Any]]:
        """
        Convert list of Python functions or schemas to Gemini API tool format.
        
        Args:
            functions: List of functions or schemas to convert
            
        Returns:
            List of function declarations in API format
        """
        tools = []
        for func in functions:
            if isinstance(func, dict):
                # Already a schema definition
                tools.append(func)
            elif callable(func):
                # Convert function to schema
                tools.append(GeminiUtils.python_to_function(func))
            else:
                logger.warning(f"Ignoring unsupported function type: {type(func)}")
        return tools
    
    @staticmethod
    def extract_function_calls(response: Any) -> List[Dict[str, Any]]:
        """
        Extract function calls from model response.
        
        Args:
            response: Model response
            
        Returns:
            List of function calls
        """
        # Parse response if it's a string
        if isinstance(response, str):
            try:
                response_data = json.loads(response)
            except json.JSONDecodeError:
                # Not JSON, no function calls
                return []
        elif isinstance(response, dict):
            response_data = response
        else:
            return []
        
        # Check for function calls in different formats
        function_calls = []
        
        # Format 1: functionCalls array
        if "functionCalls" in response_data:
            return response_data["functionCalls"]
            
        # Format 2: function_calls array
        if "function_calls" in response_data:
            return response_data["function_calls"]
            
        # Format 3: functionCall in candidates
        if "candidates" in response_data:
            for candidate in response_data.get("candidates", []):
                if isinstance(candidate, dict) and "functionCall" in candidate:
                    function_calls.append(candidate["functionCall"])
                    
        # Format 4: toolCalls in various schemas
        tool_calls = response_data.get("toolCalls", []) or response_data.get("tool_calls", [])
        if tool_calls:
            for tool in tool_calls:
                if isinstance(tool, dict) and "function" in tool:
                    function_info = tool["function"]
                    # Convert to standard format
                    function_call = {
                        "name": function_info.get("name"),
                        "args": function_info.get("arguments", {})
                    }
                    
                    # Parse args if it's a string
                    if isinstance(function_call["args"], str):
                        try:
                            function_call["args"] = json.loads(function_call["args"])
                        except json.JSONDecodeError:
                            function_call["args"] = {}
                            
                    function_calls.append(function_call)
        
        return function_calls